In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
from scipy.special import softmax

In [ ]:
# Cargar modelo preentrenado
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
# Etiquetas del modelo
labels = ['negative', 'neutral', 'positive']

In [ ]:
# Función para predecir sentimiento
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = softmax(logits.numpy()[0])
    return labels[np.argmax(probs)]

In [ ]:
# Cargar tus datos procesados
comments = pd.read_csv("data/processed/comments_processed.csv")
comments = comments.dropna(subset=['body'])

#repetir con los demas datasets

In [ ]:
# Predecir sentimiento en un subconjunto (por eficiencia)
comments["sentiment"] = comments["body"].apply(lambda x: predict_sentiment(x))

In [ ]:
# Guardar dataset con etiquetas
comments.to_csv("data/processed/comments_labeled.csv", index=False)